In [1]:
import pandas as pd
import os

In [29]:
import pandas as pd
import re

df = pd.read_csv('subsidy_amount.csv', header=[0, 1])

new_columns = []
current_model = None

for col in df.columns:
    upper = str(col[0]).strip()
    lower = str(col[1]).strip().replace('\n', ' ')
    
    if not upper or 'Unnamed' in upper:
        if current_model:
            new_columns.append(f"{current_model}_{lower}")
        else:
            new_columns.append(lower)
    else:
        current_model = upper
        new_columns.append(f"{upper}_{lower}")

df.columns = new_columns

# '지역_군/시'를 '지역'으로 변경
if '지역_군/시' in df.columns:
    df.rename(columns={'지역_군/시': '지역'}, inplace=True)

# 불필요한 컬럼 제거
columns_to_drop = []
if 'Unnamed: 0_level_1' in df.columns:
    columns_to_drop.append('Unnamed: 0_level_1')
if '지역_도' in df.columns:
    columns_to_drop.append('지역_도')

if columns_to_drop:
    df.drop(columns=columns_to_drop, inplace=True)

if '지역_도' in df.columns:
    df = df[~df['지역_도'].isin(['공단', '공단 리스금액'])]
elif '도' in df.columns:
    df = df[~df['도'].isin(['공단', '공단 리스금액'])]

# 지역명 변경
df['지역'] = df['지역'].replace({
    '한국환경공단(법인, 개사, 개인)': '공단_일반',
    '공단 리스금액': '공단_리스'
})

# 보조금 금액 컬럼에서 한글 포함된 값 제거
for col in df.columns:
    if col != '지역':
        df[col] = df[col].apply(lambda x: '' if pd.notna(x) and re.search(r'[가-힣]', str(x)) else x)

df.to_excel('subsidy_amount_cleaned.xlsx', index=False)

In [31]:
df = pd.read_excel('subsidy_amount_cleaned.xlsx')

print('컬럼 수:', len(df.columns))
print('\n컬럼명:')
for i, col in enumerate(df.columns, 1):
    print(f'{i}. {col}')

print(f'\n총 {len(df.columns)}개 컬럼')
print('행 수:', len(df))
print('\n첫 행 샘플:')
print(df.iloc[0].to_dict())
print('\n데이터 타입:')
print(df.dtypes)

컬럼 수: 13

컬럼명:
1. 지역
2. Model 3_RWD 2024
3. Model 3_RWD 2025
4. Model 3_LR
5. Model 3_LR (배터리변경)
6. Model 3_P
7. Model Y_RWD
8. Model Y_NEW RWD
9. Model Y_NEW LR (런치포함)
10. Model Y_LR (배터리변경)
11. Model Y_LR 19
12. Model Y_LR 20
13. Model Y_P

총 13개 컬럼
행 수: 163

첫 행 샘플:
{'지역': '공단_일반', 'Model 3_RWD 2024': 183, 'Model 3_RWD 2025': 186.0, 'Model 3_LR': 202.0, 'Model 3_LR (배터리변경)': nan, 'Model 3_P': 187.0, 'Model Y_RWD': 201.0, 'Model Y_NEW RWD': 188.0, 'Model Y_NEW LR (런치포함)': 207.0, 'Model Y_LR (배터리변경)': 210.0, 'Model Y_LR 19': 202.0, 'Model Y_LR 20': 184, 'Model Y_P': 191.0}

데이터 타입:
지역                        object
Model 3_RWD 2024           int64
Model 3_RWD 2025         float64
Model 3_LR               float64
Model 3_LR (배터리변경)       float64
Model 3_P                float64
Model Y_RWD              float64
Model Y_NEW RWD          float64
Model Y_NEW LR (런치포함)    float64
Model Y_LR (배터리변경)       float64
Model Y_LR 19            float64
Model Y_LR 20              int64
Model Y_P     

In [33]:
import pymysql
import sys
sys.path.append('..')
from core.sql_manager import DB_CONFIG

# Excel 파일 읽기
df = pd.read_excel('subsidy_amount_cleaned.xlsx')

# 컬럼명 매핑 (Excel 컬럼명 -> MySQL 컬럼명)
column_mapping = {
    '지역': 'region',
    'Model 3_RWD 2024': 'model_3_rwd_2024',
    'Model 3_RWD 2025': 'model_3_rwd_2025',
    'Model 3_LR': 'model_3_lr',
    'Model 3_LR (배터리변경)': 'model_3_lr_battery_change',
    'Model 3_P': 'model_3_p',
    'Model Y_RWD': 'model_y_rwd',
    'Model Y_NEW RWD': 'model_y_new_rwd',
    'Model Y_NEW LR (런치포함)': 'model_y_new_lr_launch',
    'Model Y_LR (배터리변경)': 'model_y_lr_battery_change',
    'Model Y_LR 19': 'model_y_lr_19',
    'Model Y_LR 20': 'model_y_lr_20',
    'Model Y_P': 'model_y_p'
}

# 만원 단위를 원 단위로 변환하는 함수
def convert_to_won(value):
    if pd.isna(value) or value == '':
        return None
    try:
        # 숫자로 변환 후 10000 곱하기
        return int(float(value) * 10000)
    except (ValueError, TypeError):
        return None

# 데이터 변환
df_insert = df.copy()
for col in df.columns:
    if col != '지역':
        df_insert[col] = df_insert[col].apply(convert_to_won)

# 컬럼명 변경
df_insert.rename(columns=column_mapping, inplace=True)

# MySQL에 삽입
connection = pymysql.connect(**DB_CONFIG)
try:
    with connection.cursor() as cursor:
        # 기존 데이터 삭제 (선택사항)
        # cursor.execute("TRUNCATE TABLE subsidy_amounts")
        
        # 데이터 삽입
        insert_query = """
        INSERT INTO subsidy_amounts (
            region, model_3_rwd_2024, model_3_rwd_2025, model_3_lr, 
            model_3_lr_battery_change, model_3_p, model_y_rwd, model_y_new_rwd,
            model_y_new_lr_launch, model_y_lr_battery_change, model_y_lr_19,
            model_y_lr_20, model_y_p
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """
        
        for _, row in df_insert.iterrows():
            # NaN 값을 None으로 변환
            def to_none_if_nan(val):
                return None if pd.isna(val) else val
            
            cursor.execute(insert_query, (
                row['region'],
                to_none_if_nan(row['model_3_rwd_2024']),
                to_none_if_nan(row['model_3_rwd_2025']),
                to_none_if_nan(row['model_3_lr']),
                to_none_if_nan(row['model_3_lr_battery_change']),
                to_none_if_nan(row['model_3_p']),
                to_none_if_nan(row['model_y_rwd']),
                to_none_if_nan(row['model_y_new_rwd']),
                to_none_if_nan(row['model_y_new_lr_launch']),
                to_none_if_nan(row['model_y_lr_battery_change']),
                to_none_if_nan(row['model_y_lr_19']),
                to_none_if_nan(row['model_y_lr_20']),
                to_none_if_nan(row['model_y_p'])
            ))
        
        connection.commit()
        print(f"총 {len(df_insert)}개 행이 성공적으로 삽입되었습니다.")
        
except Exception as e:
    connection.rollback()
    print(f"오류 발생: {e}")
    raise
finally:
    connection.close()



총 163개 행이 성공적으로 삽입되었습니다.


In [34]:
import pymysql
import sys
sys.path.append('..')
from core.sql_manager import DB_CONFIG

connection = pymysql.connect(**DB_CONFIG)
try:
    with connection.cursor() as cursor:
        # subsidy_applications 테이블에서 model 컬럼의 value_count 확인
        query = """
        SELECT model, COUNT(*) as count 
        FROM subsidy_applications 
        GROUP BY model 
        ORDER BY count DESC
        """
        
        cursor.execute(query)
        results = cursor.fetchall()
        
        print("subsidy_applications 테이블의 model 컬럼 value_count:")
        print("-" * 50)
        for row in results:
            model = row[0] if row[0] is not None else "NULL"
            count = row[1]
            print(f"{model}: {count:,}개")
        
        print("-" * 50)
        total = sum(row[1] for row in results)
        print(f"총 레코드 수: {total:,}개")
        
except Exception as e:
    print(f"오류 발생: {e}")
    raise
finally:
    connection.close()

subsidy_applications 테이블의 model 컬럼 value_count:
--------------------------------------------------
NULL: 14,100개
Model Y R: 23개
Model Y L: 12개
Model 3 R: 5개
--------------------------------------------------
총 레코드 수: 14,140개
